In [366]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
import numpy as np

In [2]:
url1 = "https://redplanetscience.com"

In [3]:
url2 = "https://galaxyfacts-mars.com/"

In [209]:
url3 = "https://marshemispheres.com/"

In [5]:
url4 = "https://spaceimages-mars.com/"

This calls pandas to read the tables in second declared html to be cleaned and pushed to database.

In [213]:
tables = pd.read_html(url2)
type(tables)

list

In [218]:
df1 = tables[0]
df2 = tables[1]
df1.head()

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [358]:
df1 = df1.rename(columns={0:" ",1:'Mars',2:"Earth"})
df1

,,Mars,Earth
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [219]:
df2.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [8]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', ** executable_path, headless=False)

In [87]:
browser.visit(url1)

In [88]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [89]:
titles = []
paragraphs = []

In [90]:
title = soup.find_all('div', class_='content_title')

In [98]:
for i in title:
    string = i.text.strip()
    titles.append(string)

In [102]:
paragraph = soup.find_all('div', class_='article_teaser_body')

In [103]:
for x in paragraph:
    paragraphs.append(x.text.strip())

In [381]:
print(titles[0])
print(paragraphs[0])

NASA's Mars Helicopter Attached to Mars 2020 Rover
The helicopter will be first aircraft to perform flight tests on another planet.


In [382]:
latest_title = [titles[0]]

In [383]:
latest_paragraph = [paragraphs[0]]

In [384]:
print(latest_news)

["NASA's Mars Helicopter Attached to Mars 2020 Rover"
 'The helicopter will be first aircraft to perform flight tests on another planet.']


In [385]:
latest_title = np.array(latest_title)
latest_paragraph = np.array(latest_paragraph)

In [387]:
latest_paragraph

array(['The helicopter will be first aircraft to perform flight tests on another planet.'],
      dtype='<U80')

In [389]:
latest_news = pd.DataFrame({"title":latest_title,"paragraph":latest_paragraph})

In [390]:
latest_news

,title,paragraph
0,NASA's Mars Helicopter Attached to Mars 2020 R...,The helicopter will be first aircraft to perfo...


In [106]:
browser.visit(url4)

In [107]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [173]:
try:
    browser.links.find_by_partial_text('FULL IMAGE').click()
    img = soup.find('a', class_='showimg fancybox-thumbs')
except ElementDoesNotExist:
    print("Scraping Complete")

In [359]:
print(img)

https://marshemispheres.com/images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png


In [196]:
img = str(img)
# print(img)

In [206]:
img = "https://spaceimages-mars.com/"+img.split('"')[3]

In [397]:
img = np.array(img)

In [406]:
featured_img = pd.DataFrame({"featured_img":img}, index = [0])

In [407]:
featured_img

,featured_img
0,https://marshemispheres.com/images/4e59980c1c5...


In [210]:
browser.visit(url3)

In [211]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [284]:
item = soup.find_all('div', class_='item')

In [244]:
item = str(item)

In [331]:
title_list = []
img_list = []

In [332]:
for pic in item:
    title = ((str(pic.h3)).split("<h3>")[1])
    title = (title.split("</h3>")[0])
    title_list.append(title)
    img = "https://marshemispheres.com/"+str(pic).split('"')[11]
    img_list.append(img)

In [335]:
print(img_list[0])
print(title_list[0])

https://marshemispheres.com/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png
Cerberus Hemisphere Enhanced


In [392]:
# lists = zip(img_list,title_list)
img_list = np.array(img_list)
title_list = np.array(title_list)

In [421]:
hemispheres_df = pd.DataFrame({"images":img_list,"titles":title_list})

In [422]:
hemispheres_df

,images,titles
0,https://marshemispheres.com/images/39d32665534...,Cerberus Hemisphere Enhanced
1,https://marshemispheres.com/images/08eac6e22c0...,Schiaparelli Hemisphere Enhanced
2,https://marshemispheres.com/images/55a0a1e2796...,Syrtis Major Hemisphere Enhanced
3,https://marshemispheres.com/images/4e59980c1c5...,Valles Marineris Hemisphere Enhanced


In [336]:
browser.quit()

In [416]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [417]:
#Creates/Defines database in Mongo
db = client.mars_db

In [418]:
#Define tables in database
facts = db.facts
news = db.news
featured = db.featured
hemispheres = db.hemispheres

In [423]:
#Creates and inserts all dataframes into their tables
news.insert_many(latest_news.to_dict('records'))
facts.insert_many(df1.to_dict('records'))
featured.insert_many(featured_img.to_dict('records'))
hemispheres.insert_many(hemispheres_df.to_dict('records'))